### Run docker qdrant

In [8]:
!docker run -p "6333:6333" -p "6334:6334" --name "rag-openai-qdrant" --rm -d qdrant/qdrant:latest

docker: Error response from daemon: Conflict. The container name "/rag-openai-qdrant" is already in use by container "f144979bd6bc485a60615cd3cd0945f62cd30b8269d3405ef9f3459b26bd37ab". You have to remove (or rename) that container to be able to reuse that name.
See 'docker run --help'.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [9]:
import qdrant_client

client = qdrant_client.QdrantClient("http://localhost:6333", prefer_grpc=True)
client.get_collections()

CollectionsResponse(collections=[])

### Import knowledge

In [15]:
client.add(
    collection_name="knowledge-base",
    documents=[
        "Qdrant is a vector database & vector similarity search engine. It deploys as an API service providing search for the nearest high-dimensional vectors. With Qdrant, embeddings or neural network encoders can be turned into full-fledged applications for matching, searching, recommending, and much more!",
        "Docker helps developers build, share, and run applications anywhere — without tedious environment configuration or management.",
        "PyTorch is a machine learning framework based on the Torch library, used for applications such as computer vision and natural language processing.",
        "MySQL is an open-source relational database management system (RDBMS). A relational database organizes data into one or more data tables in which data may be related to each other; these relations help structure the data. SQL is a language that programmers use to create, modify and extract data from the relational database, as well as control user access to the database.",
        "NGINX is a free, open-source, high-performance HTTP server and reverse proxy, as well as an IMAP/POP3 proxy server. NGINX is known for its high performance, stability, rich feature set, simple configuration, and low resource consumption.",
        "FastAPI is a modern, fast (high-performance), web framework for building APIs with Python 3.7+ based on standard Python type hints.",
        "SentenceTransformers is a Python framework for state-of-the-art sentence, text and image embeddings. You can use this framework to compute sentence / text embeddings for more than 100 languages. These embeddings can then be compared e.g. with cosine-similarity to find sentences with a similar meaning. This can be useful for semantic textual similar, semantic search, or paraphrase mining.",
        "The cron command-line utility is a job scheduler on Unix-like operating systems. Users who set up and maintain software environments use cron to schedule jobs (commands or shell scripts), also known as cron jobs, to run periodically at fixed times, dates, or intervals.",
    ]
)

['2a9c54fb217842ca8b1efc7f478f8d9d',
 'a1e88b225a094c53a4e48cf4e1a4d2f5',
 '4f7e2175aee84b1a923634718fa4e923',
 '418393db4ae44b5ead7e7c7475d573f0',
 '9a8e668d74e44b4dbf153348922103ae',
 'a3e4d6027ae8469391ba7ea09c62a88b',
 '4394b8db7ffd4edd97768cfc87838652',
 '888e186501194ac08779473232f85e66']

In [11]:
prompt = """
What tools should I need to use to build a web service using vector embeddings for search?
"""

#### Setting openai key

In [12]:
import os

# Fill the environmental variable with your own OpenAI API key
# See: https://platform.openai.com/account/api-keys
os.environ["OPENAI_API_KEY"] = ""

In [13]:
import openai

completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo", 
    messages=[
        {"role": "user", "content": prompt},
    ]
)
print(completion["choices"][0]["message"]["content"])

To build a web service using vector embeddings for search, you will need the following tools:

1. Programming Language: You can use Python, Java, or any other language that supports the libraries and tools needed for working with vector embeddings.

2. Vector Embeddings: You will need a library or tool that can generate vector embeddings for your data. Popular options include Word2Vec, GloVe, FastText, or using pre-trained embeddings such as BERT or Universal Sentence Encoder.

3. Web Framework: You will need a web framework such as Flask or Django to build the web service and handle HTTP requests.

4. Database: You may need a database to store and retrieve the vector embeddings and the associated data for search. Options include SQL databases like PostgreSQL or NoSQL databases like MongoDB.

5. Search Engine: You may need a search engine such as Elasticsearch or Solr to efficiently search and retrieve results based on the vector embeddings.

6. API: You will need to create an API to e

#### Build a function to facilitate querying

In [20]:
def rag(question: str, n_points: int = 3) -> str:
    results = client.query(
        collection_name="knowledge-base",
        query_text=question,
        limit=n_points,
    )

    context = "\n".join(r.document for r in results)

    metaprompt = f"""
    You are a software architect. 
    Answer the following question using the provided context. 
    If you can't find the answer, do not pretend you know it, but answer "I don't know".
    
    Question: {question.strip()}
    
    Context: 
    {context.strip()}
    
    Answer:
    """

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": metaprompt},
        ],
        timeout=10.0,
    )
    return completion["choices"][0]["message"]["content"]

In [21]:
rag("What can the stack for a web api look like?")

'The stack for a web API using FastAPI with Python could potentially include Docker for containerization and NGINX as a high-performance reverse proxy server. The specific components and technologies used in the stack may vary depending on the requirements and preferences of the project.'

### Try another data knowledge

In [24]:
client.add(
    collection_name="knowledge-base",
    documents=[
        "Zhang Ziyi: Renowned for her roles in internationally acclaimed films like \"Crouching Tiger, Hidden Dragon,\" \"Hero,\" and \"Memoirs of a Geisha,\" Zhang Ziyi is one of China's most recognizable and celebrated actresses. She has received numerous awards for her performances and is known for her versatility and emotional depth on screen.",

        "Gong Li: Often hailed as one of the greatest actresses in Chinese cinema, Gong Li has starred in many iconic films directed by Zhang Yimou, including \"Raise the Red Lantern,\" \"Ju Dou,\" and \"To Live.\" Her powerful performances and captivating presence have earned her international acclaim and accolades.",

        "Zhou Xun: With a career spanning over two decades, Zhou Xun has established herself as a versatile actress known for her ability to portray a wide range of characters. She has starred in films such as \"Perhaps Love,\" \"The Banquet,\" and \"The Great Magician,\" showcasing her talent and versatility."

        "Fan Bingbing: As one of China's highest-paid actresses, Fan Bingbing has achieved widespread recognition both domestically and internationally. She has appeared in blockbuster films like \"X-Men: Days of Future Past\" and \"The White Haired Witch of Lunar Kingdom,\" as well as critically acclaimed dramas such as \"Cell Phone\" and \"Lost in Beijing.\""

        "Li Bingbing: Known for her roles in both Chinese and Hollywood productions, Li Bingbing has starred in films like \"The Forbidden Kingdom,\" \"Transformers: Age of Extinction,\" and \"Detective Dee and the Mystery of the Phantom Flame.\" She is admired for her beauty, talent, and versatility as an actress."

        "Tang Wei: Tang Wei gained international fame for her performance in Ang Lee's film \"Lust, Caution,\" for which she won the Best New Performer award at the Golden Horse Awards. She has since appeared in various Chinese and international productions, showcasing her acting prowess and captivating screen presence."


        "Liu Yifei: Best known for her role as Mulan in Disney's live-action adaptation of the animated classic, Liu Yifei has been a prominent figure in Chinese cinema for many years. She has starred in various historical dramas, romantic comedies, and action films, establishing herself as a versatile and talented actress."

        "Yang Mi: With a successful career spanning film, television, and music, Yang Mi has become one of China's most popular and influential actresses. She has appeared in a wide range of genres, from romantic dramas to fantasy epics, and has garnered a large fan following both in China and internationally."

        "Bailu, also known as 白鹿 (Bai Lu) in Mandarin, is a Chinese actress known for her performances in various television dramas and films. She gained recognition for her roles in popular Chinese dramas such as \"The Legends\" (招摇) and \"Arsenal Military Academy\" (烈火军校). Bailu has garnered praise for her acting skills and versatility, portraying a wide range of characters with depth and emotion."
    ]
)

['948272d88e57400bb6369f93270435d0',
 '7aba02da41c14d8caf6695bbf1ceca04',
 '45571bbf534b4a0abf56e059999125ce']

In [25]:
rag("Who is Bailu?")

'Bailu is a Chinese actress known for her performances in various television dramas and films.'

In [27]:
rag("tell me about name some of actor in China?")

'Zhou Xun, Fan Bingbing, Li Bingbing, Tang Wei, Liu Yifei, Yang Mi, Bailu, Zhang Ziyi, and Gong Li are some of the prominent actors in China.'

#### clear enviroment

In [28]:
!docker kill rag-openai-qdrant
!docker rm rag-openai-qdrant

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


rag-openai-qdrant
Error: No such container: rag-openai-qdrant


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
